In [2]:
#Import the geemap and google earth engine module 
import geemap
import ee

In [3]:
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [63]:
#Select Study Area

feature = Map.draw_last_feature
aoi = feature.geometry()

# Define time interval for the Sentinel - 1 data

#Before Flood (Start Date)
before_start='2021-07-07'
before_end='2021-07-12'

#After Flood (End Date)
after_start='2021-07-14'
after_end='2021-07-16'

#Parameters for SAR

polarization ="VH";            #Preferred polarization for flood mapping, but you can also select 'VV'

pass_direction="DESCENDING";   #Or ASCENDING, consider changing this parameter if your image collection is empty

difference_threshold=1.25;     #Threshold to be applied on the difference image, leave default

#Dont make changes in the below section

#Load and Select Sentinel-1 data by predefined level
# Load and filter Sentinel-1 GRD data by predefined parameters
collection= ee.ImageCollection('COPERNICUS/S1_GRD') \
  .filter(ee.Filter.eq('instrumentMode','IW')) \
  .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
  .filter(ee.Filter.eq('orbitProperties_pass',pass_direction)) \
  .filter(ee.Filter.eq('resolution_meters',10)) \
  .filterBounds(aoi) \
  .select(polarization)

# Select images by predefined dates
before_collection = collection.filterDate(before_start, before_end)
after_collection = collection.filterDate(after_start,after_end)

# Create a mosaic of selected tiles and clip to study area
before = before_collection.mosaic().clip(aoi)
after = after_collection.mosaic().clip(aoi)

# Apply reduce the radar speckle by smoothing
smoothing_radius = 50
before_filtered = before.focal_mean(smoothing_radius, 'circle', 'meters')
after_filtered = after.focal_mean(smoothing_radius, 'circle', 'meters')

#------------------------------- FLOOD EXTENT CALCULATION -------------------------------#

# Calculate the difference between the before and after images
difference = after_filtered.divide(before_filtered)

# Apply the predefined difference-threshold and create the flood extent mask
threshold = difference_threshold
difference_binary = difference.gt(threshold)

# Refine flood result using additional datasets

# Include JRC layer on surface water seasonality to mask flood pixels from areas
# of "permanent" water (where there is water > 10 months of the year)
swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
swater_mask = swater.gte(10).updateMask(swater.gte(10))

#Flooded layer where perennial water bodies (water > 10 mo/yr) is assigned a 0 value
flooded_mask = difference_binary.where(swater_mask,0)
# final flooded area without pixels in perennial waterbodies
flooded = flooded_mask.updateMask(flooded_mask)

# Compute connectivity of pixels to eliminate those connected to 8 or fewer neighbours
# This operation reduces noise of the flood extent product
connections = flooded.connectedPixelCount()
flooded = flooded.updateMask(connections.gte(8))

# Mask out areas with more than 5 percent slope using a Digital Elevation Model
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM')
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
flooded = flooded.updateMask(slope.lt(5))

# Calculate flood extent area
# Create a raster layer containing the area information of each pixel
flood_pixelarea = flooded.select(polarization) \
  .multiply(ee.Image.pixelArea())

# Sum the areas of flooded pixels
# default is set to 'bestEffort: True' in order to reduce compuation time, for a more
# accurate result set bestEffort to False and increase 'maxPixels'.

flood_stats= flood_pixelarea.reduceRegion(
        reducer= ee.Reducer.sum(),
        geometry= aoi,
        scale= 30,
        bestEffort= True
)


#Display
#Before and after flood SAR mosaic
Map.setCenter('aoi', 10)
Map.addLayer(before_filtered, {'min':-25, 'max':0}, 'Before Flood',0)
Map.addLayer(after_filtered, {'min':-25, 'max':0}, 'After Flood',1)
Map.addLayer(difference,{'min':0, 'max':2},"Difference Layer",0)
Map.addLayer(flooded,{'palette':"0000FF"},'Flooded areas')